In [1]:
import pandas as pd
import numpy as np
from jarrett_cleaning import clean_history

In [2]:
dan = pd.read_csv("NetflixViewingHistoryDan.csv")

In [10]:
dan.head(20)

,Title,Date
0,Big Fish,3/28/22
1,Monsters vs. Aliens,3/19/22
2,Brand New Cherry Flavor: Limited Series: Tadpo...,3/10/22
3,Brand New Cherry Flavor: Limited Series: I Exist,3/10/22
4,Starship Troopers,3/9/22
5,I Think You Should Leave with Tim Robinson: Se...,3/7/22
6,I Think You Should Leave with Tim Robinson: Se...,3/7/22
7,I Think You Should Leave with Tim Robinson: Se...,3/6/22
8,You vs. Wild: Season 1: Operation Jungle Rescu...,2/26/22
9,You vs. Wild: Season 1: Operation Jungle Rescu...,2/26/22


In [5]:
netflix = pd.read_csv("netflix_titles.csv")

In [6]:
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [7]:
import difflib

In [17]:
best_matches = []
for i in range(len(dan["Title"])):
    best_matches.append(difflib.get_close_matches(dan['Title'][i], netflix['title'], 10))

KeyboardInterrupt: 

In [16]:
best_matches

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [32]:
best_matches = dan['Title'].apply(lambda x: difflib.get_close_matches(x, netflix['title']))

In [33]:
best_matches

0                    [Big Time Rush, Big Time, Big Kill]
1      [Monster Island, Monster House, Super Monsters...
2                                                     []
3                              [Brand New Cherry Flavor]
4      [Starship Troopers: Traitor of Mars, Star Trek...
                             ...                        
973                                         [Battlefish]
974                         [Blackfish, Black, Backfire]
975               [The Emigrant, The Giant, The Reliant]
976                                    [Elisa & Marcela]
977                        [Red Dawn, Red Joan, Red Dot]
Name: Title, Length: 978, dtype: object

In [39]:
genres = []
for i in range(len(best_matches)):
    item_genres = []
    for j in range(len(best_matches[i])):
        item_genres.append(netflix['listed_in'].values[netflix['title'] == best_matches[i][j]])
    genres.append(item_genres)

In [40]:
genres

[[array(["Kids' TV"], dtype=object),
  array(['Documentaries'], dtype=object),
  array(['Action & Adventure'], dtype=object)],
 [array(['Children & Family Movies'], dtype=object),
  array(['Children & Family Movies, Comedies'], dtype=object),
  array(['Children & Family Movies'], dtype=object)],
 [],
 [array(['TV Dramas, TV Horror, TV Mysteries'], dtype=object)],
 [array(['Action & Adventure, Sci-Fi & Fantasy'], dtype=object),
  array(['Action & Adventure, Sci-Fi & Fantasy'], dtype=object),
  array(["Kids' TV"], dtype=object)],
 [array(['TV Comedies'], dtype=object)],
 [],
 [array(['TV Comedies'], dtype=object)],
 [],
 [],
 [],
 [],
 [],
 [array(['British TV Shows, Reality TV'], dtype=object),
  array(['British TV Shows, Reality TV'], dtype=object),
  array(['British TV Shows, Reality TV'], dtype=object)],
 [array(['British TV Shows, Reality TV'], dtype=object),
  array(['British TV Shows, Reality TV'], dtype=object),
  array(['British TV Shows, Reality TV'], dtype=object)],
 [array(['

In [48]:
final_genres = []
for i in range(len(genres)):
    movie = []
    for j in range(len(genres[i])):
        for k in range(len(genres[i][j])):
            movie.append(genres[i][j][k])
            #print(genres[i][j][k])
    final_genres.append(movie)

In [49]:
final_genres

[["Kids' TV", 'Documentaries', 'Action & Adventure'],
 ['Children & Family Movies',
  'Children & Family Movies, Comedies',
  'Children & Family Movies'],
 [],
 ['TV Dramas, TV Horror, TV Mysteries'],
 ['Action & Adventure, Sci-Fi & Fantasy',
  'Action & Adventure, Sci-Fi & Fantasy',
  "Kids' TV"],
 ['TV Comedies'],
 [],
 ['TV Comedies'],
 [],
 [],
 [],
 [],
 [],
 ['British TV Shows, Reality TV',
  'British TV Shows, Reality TV',
  'British TV Shows, Reality TV'],
 ['British TV Shows, Reality TV',
  'British TV Shows, Reality TV',
  'British TV Shows, Reality TV'],
 ['British TV Shows, Reality TV',
  'British TV Shows, Reality TV',
  'British TV Shows, Reality TV'],
 ['British TV Shows, Reality TV',
  'British TV Shows, Reality TV',
  'British TV Shows, Reality TV'],
 ['Action & Adventure, Sci-Fi & Fantasy', 'Dramas', "Kids' TV, TV Comedies"],
 ['British TV Shows, Reality TV',
  'British TV Shows, Reality TV',
  'British TV Shows, Reality TV'],
 ['British TV Shows, Reality TV',
  'Brit

In [50]:
all_genres = []
for movie in final_genres:
    all_genres += movie

In [54]:
all_genres = pd.Series(all_genres)
all_genres.value_counts()

Stand-Up Comedy                                      88
Action & Adventure                                   48
TV Comedies                                          39
Docuseries                                           34
British TV Shows, Reality TV                         33
                                                     ..
Action & Adventure, Cult Movies, Sci-Fi & Fantasy     1
Classic Movies, Cult Movies, Dramas                   1
Music & Musicals                                      1
Classic Movies, Documentaries                         1
Action & Adventure, Cult Movies                       1
Length: 179, dtype: int64

In [24]:
matches = difflib.get_close_matches(dan['Title'][13], netflix['title'])

In [25]:
dan["Title"][13]

'The Great British Baking Show: Collection 6: Cake Week'

In [26]:
matches

['The Great British Baking Show: Holidays',
 'The Great British Baking Show',
 'The Great British Baking Show: Masterclass']

In [30]:
for match in matches:
    print(difflib.SequenceMatcher(a=dan["Title"][13], b=match).ratio())

0.7526881720430108
0.6987951807228916
0.6875


In [31]:
print(difflib.SequenceMatcher(a=dan["Title"][13], b=matches).ratio())

0.0
